# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tbanerje\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tbanerje\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tbanerje\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import precision_recall_fscore_support
import pickle
import warnings

# to ignore warnings
warnings.filterwarnings("ignore")

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', con = engine)
X = df['message']
Y = df.iloc[:,4:]

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    #sanitize - replace non-alphanumeric char with space, and normalize
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #remove stop words
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    
    #lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = []
    for word in words:
        token = lemmatizer.lemmatize(word).lower().strip()
        tokens.append(token)
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(OneVsRestClassifier(RandomForestClassifier())))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 40)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...te=None, verbose=0,
            warm_start=False),
          n_jobs=None),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [33]:
# Get results and add them to a dataframe.
def compute_results(y_test, y_pred):
    results = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    num = 0
    for category in y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[category], y_pred[:,num], average='weighted')
        results.set_value(num+1, 'Category', category)
        results.set_value(num+1, 'f_score', f_score)
        results.set_value(num+1, 'precision', precision)
        results.set_value(num+1, 'recall', recall)
        num += 1
    
    aggregated_results = pd.DataFrame(columns=['Aggregated_Result_type', 'Value'])
    line = pd.DataFrame({'Aggregated_Result_type': 'f_score', 'Value': results['f_score'].mean()}, index=[0])
    aggregated_results = aggregated_results.append(line)
    line = pd.DataFrame({'Aggregated_Result_type': 'precision', 'Value': results['precision'].mean()}, index=[1])
    aggregated_results = aggregated_results.append(line)
    line = pd.DataFrame({'Aggregated_Result_type': 'recall', 'Value': results['recall'].mean()}, index=[2])
    aggregated_results = aggregated_results.append(line)
    
    return results, aggregated_results

In [22]:
Y_pred = pipeline.predict(X_test)

In [34]:
results, aggregated_results = compute_results(Y_test, Y_pred)
aggregated_results

,Aggregated_Result_type,Value
0,f_score,0.932768
1,precision,0.935589
2,recall,0.945223


In [35]:
results

,Category,f_score,precision,recall
1,related,0.804445,0.812388,0.820557
2,request,0.870921,0.880658,0.885774
3,offer,0.993997,0.992007,0.995995
4,aid_related,0.760165,0.76583,0.765256
5,medical_help,0.895936,0.892395,0.921243
6,medical_products,0.934928,0.94469,0.951945
7,search_and_rescue,0.962921,0.963689,0.97254
8,security,0.976394,0.971934,0.9836
9,military,0.958375,0.958179,0.969489
10,child_alone,1,1,1


### 6. Improve your model
Use grid search to find better parameters. 

In [24]:
parameters = {
        'vectorizer__ngram_range': ((1, 1), (1, 2)),
        'tfidf__use_idf': [True, False]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=1, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [25]:
cv.fit(X_train, Y_train)
print('Fitted.')

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 54.1min finished


Fitted.


In [36]:
Y_pred = cv.predict(X_test)

In [37]:
new_results, new_aggregated_results = compute_results(Y_test, Y_pred)
new_aggregated_results

,Aggregated_Result_type,Value
0,f_score,0.928979
1,precision,0.932781
2,recall,0.942681


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [39]:
pickle.dump(cv, open('disaster_resp_classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.